In [1]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

import sys; sys.path.insert(0, r'C:\Users\Lukas\Documents\projects\invert')
import mne
import pickle as pkl

from time import time

from invert.forward import get_info, create_forward_model
from invert.simulate import generator
from invert import Solver

# Load Files

In [2]:
fwd = mne.read_forward_solution("forward_model/64ch_ico3-fwd.fif", verbose=0)
fwd = mne.convert_forward_solution(fwd, force_fixed=True)
fn = "forward_model/64ch_info.pkl"
with open(fn, 'rb') as f:
    info = pkl.load(f)

    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Changing to fixed-orientation forward solution with surface-based source orientations...
    [done]


# Simulate Data

## Settings

In [6]:
from copy import deepcopy
generator_args_single = dict(
    use_cov = False, 
    return_mask = False, 
    n_sources = (1, 10), 
    n_orders = (0,0),
    snr_range = (0.1, 100),
    amplitude_range = (0.001, 1),
    batch_repetitions = 1,
    batch_size = 500,
    scale_data = False,
    n_timepoints = 20,
    beta_range = (0, 3),
    return_info = True,
    inter_source_correlation=(0, 1))

generator_args_extended = deepcopy(generator_args_single)
generator_args_extended["n_orders"] = (1, 3)

generator_args = dict(single=generator_args_single, extended=generator_args_extended)
solver_dicts = [
    {
        "name": "FLEX-AP",
        "make_args": {
            "n_orders": generator_args_extended["n_orders"][1],
            "refine_solution": True,
            "stop_crit": 0.95,
        },
        "apply_args": {
            
        },
        "recompute_make": True
    },
    {
        "name": "FLEX-MUSIC",
        "make_args": {
            "n_orders": generator_args_extended["n_orders"][1],
            "refine_solution": False,
            "stop_crit": 0.95,
        },
        "apply_args": {
            
        },
        "recompute_make": True
    },
    {
        "name": "AP",
        "make_args": {
            "n_orders": 0,
            "refine_solution": True,
            "stop_crit": 0.95,
        },
        "apply_args": {
            
        },
        "recompute_make": True
    },
    {
        "name": "RAP-MUSIC",
        "make_args": {
            "n_orders": 0,
            "refine_solution": False,
            "stop_crit": 0.95,
        },
        "apply_args": {
            
        },
        "recompute_make": True
    },
    {
        "name": "eLORETA",
        "make_args": {
            
        },
        "apply_args": {
            
        },
        "recompute_make": False
    },
    {
        "name": "Convexity Champagne",
        "make_args": {
            
        },
        "apply_args": {
            
        },
        "recompute_make": True
    },
    {
        "name": "MCMV",
        "make_args": {
            
        },
        "apply_args": {
            
        },
        "recompute_make": True
    }
]
solvers_names = [s["name"] for s in solver_dicts]

In [7]:
# Generate Samples
solvers = {solver_name: Solver(solver_name) for solver_name in solvers_names}
for sim_type, generator_args_  in generator_args.items():
    proc_time_make = dict()
    proc_time_apply = dict()
    gen_test = generator(fwd, **generator_args_)
    x_test, y_test, sim_info = gen_test.__next__()
    stc_dict = dict()

    for i_sample, (x_sample, y_sample) in enumerate(zip(x_test, y_test)):
        print("Sample ", i_sample)
        evoked = mne.EvokedArray(x_sample.T, info, tmin=0)
        for solver_dict in solver_dicts:
            solver_name = solver_dict["name"]
            make_args = solver_dict["make_args"]
            apply_args = solver_dict["apply_args"]
            recompute_make = solver_dict["recompute_make"]
            solver = solvers[solver_name]
            
            print(solver_name)
            if i_sample == 0:
                stc_dict[solver_name] = []
                proc_time_make[solver_name] = []
                proc_time_apply[solver_name] = []
                
            if recompute_make or i_sample == 0:
                start_make = time()
                solver.make_inverse_operator(fwd, evoked, alpha="auto", n=sim_info.n_sources.values[i_sample], **make_args)
                end_make = time()
            
            start_apply = time()

            stc = solver.apply_inverse_operator(evoked)
            end_apply = time()
            
            stc_dict[solver_name].append(stc)
            proc_time_make[solver_name].append(end_make - start_make)
            proc_time_apply[solver_name].append(end_apply - start_apply)
            
    fn = f"evaluation/sim_and_preds_{sim_type}.pkl"
    with open(fn, 'wb') as f:
        pkl.dump([stc_dict, x_test, y_test, sim_info, proc_time_make, proc_time_apply], f)

Using Diffusion Smoothing on Graph
Sample  0
FLEX-AP
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
FLEX-MUSIC
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
AP
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pi

# Save solvers

In [8]:
import os
for solver in solvers.values():
    path = f"saved_solvers/{solver.name}"
    if ~os.path.isdir(path):
        os.mkdir(path)
    solver.save(path)

FileExistsError: [WinError 183] Eine Datei kann nicht erstellt werden, wenn sie bereits vorhanden ist: 'saved_solvers/Flexible Alternating Projections'

# Plot samples

In [9]:
from scipy.stats import pearsonr

pp = dict(surface='white', hemi='both', verbose=0)
sample = 0

stc_ = stc.copy()
stc_.data = y_test[sample].T
stc_.plot(**pp, brain_kwargs=dict(title="True"))

for solver, stc_list in stc_dict.items():
    stc_list[sample].plot(**pp, brain_kwargs=dict(title=solver))
    print(solver, " r = ", round(pearsonr(abs(stc_.data).mean(axis=-1), abs(stc_list[sample].data).mean(axis=-1))[0], 2))
display(sim_info.iloc[sample])

FLEX-AP  r =  0.9
FLEX-MUSIC  r =  0.9
AP  r =  0.85
RAP-MUSIC  r =  0.85
eLORETA  r =  0.26
Convexity Champagne  r =  0.95
MCMV  r =  0.16


n_sources                                       1
amplitudes                   [0.6515591198498345]
snr                                     39.822622
inter_source_correlations                0.180291
Name: 0, dtype: object

Using control points [0.         0.         0.01131048]


In [18]:
sim_info

,n_sources,amplitudes,snr,inter_source_correlations
0,9,"[0.4092431504577343, 0.010705234109414558, 0.8...",62.266083,0.777220
1,8,"[0.19272968530071355, 0.6428331006075295, 0.93...",94.957932,0.543688


Using control points [0.00271854 0.00328425 0.03182027]
